## youtube8Mのデータセット準備

とりあえずローカルでDatasetAPIで学習を開始できるようにするのが目的

### ローカルへのデータダウンロード

youtube8mのデータは以下のコマンドで取って来れます。
下のセルを実行すると全データの1/100のトレーニングデータを取得します。

環境変数からオプションを渡すような作りになってるみたい。
このpartition=2の部分がよくわからないけど、とりあえずこのままでw

In [1]:
# youtube8m のディレクトリに移動
import os
while os.getcwd().split('/')[-1] != 'youtube8m': os.chdir('..')
os.getcwd()

'/home/jupyter/ASLOpenProject/youtube8m'

In [2]:
BUCKET="asl-mixi-project-bucket"

In [3]:
%%bash 
CURRENT_DIR=$(PWD)
mkdir ./data/video/
cd ./data/video/
curl data.yt8m.org/download.py > download.py

# シャードで1/100をダウンロードするには以下を
#shard=1,100 partition=2/video/train mirror=us python download.py
#shard=1,100 partition=2/video/validate mirror=us python download.py
partition=2/video/train mirror=us python download.py
partition=2/video/validate mirror=us python download.py
cd $CURRENT_DIR

Resuming Download ...
Files remaining 0
All done. No more files to download.
Resuming Download ...
Files remaining 0
All done. No more files to download.


bash: line 1: PWD: command not found
mkdir: cannot create directory ‘./data/video/’: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4450  100  4450    0     0  11384      0 --:--:-- --:--:-- --:--:-- 11410


In [4]:
!ls ./data/video/train*.tfrecord -l | wc -l

3844


In [5]:
!ls ./data/video/train*.tfrecord -alh | head

-rw-r--r-- 1 jupyter jupyter 4.6M Dec  9 07:47 ./data/video/train0000.tfrecord
-rw-r--r-- 1 jupyter jupyter 4.6M Dec  9 07:40 ./data/video/train0001.tfrecord
-rw-r--r-- 1 jupyter jupyter 4.6M Dec  9 07:36 ./data/video/train0002.tfrecord
-rw-r--r-- 1 jupyter jupyter 4.5M Dec  9 07:51 ./data/video/train0003.tfrecord
-rw-r--r-- 1 jupyter jupyter 4.5M Dec  9 07:47 ./data/video/train0004.tfrecord
-rw-r--r-- 1 jupyter jupyter 4.7M Dec  9 07:43 ./data/video/train0005.tfrecord
-rw-r--r-- 1 jupyter jupyter 4.5M Dec  9 07:41 ./data/video/train0006.tfrecord
-rw-r--r-- 1 jupyter jupyter 4.5M Dec  9 07:44 ./data/video/train0007.tfrecord
-rw-r--r-- 1 jupyter jupyter 4.5M Dec  9 07:45 ./data/video/train0008.tfrecord
-rw-r--r-- 1 jupyter jupyter 4.6M Dec  9 07:42 ./data/video/train0009.tfrecord
ls: write error: Broken pipe


In [6]:
!ls ./data/video/validate*.tfrecord -l | wc -l

3844


In [7]:
!ls ./data/video/validate*.tfrecord -alh | head

-rw-r--r-- 1 jupyter jupyter 1.3M Dec  9 07:53 ./data/video/validate0000.tfrecord
-rw-r--r-- 1 jupyter jupyter 1.4M Dec  9 07:53 ./data/video/validate0001.tfrecord
-rw-r--r-- 1 jupyter jupyter 1.5M Dec  9 07:53 ./data/video/validate0002.tfrecord
-rw-r--r-- 1 jupyter jupyter 1.4M Dec  9 07:56 ./data/video/validate0003.tfrecord
-rw-r--r-- 1 jupyter jupyter 1.3M Dec  9 07:56 ./data/video/validate0004.tfrecord
-rw-r--r-- 1 jupyter jupyter 1.4M Dec  9 07:55 ./data/video/validate0005.tfrecord
-rw-r--r-- 1 jupyter jupyter 1.3M Dec  9 07:52 ./data/video/validate0006.tfrecord
-rw-r--r-- 1 jupyter jupyter 1.4M Dec  9 08:00 ./data/video/validate0007.tfrecord
-rw-r--r-- 1 jupyter jupyter 1.4M Dec  9 07:54 ./data/video/validate0008.tfrecord
-rw-r--r-- 1 jupyter jupyter 1.2M Dec  9 07:57 ./data/video/validate0009.tfrecord
ls: write error: Broken pipe


### storageへのコピー

必要があれば実行。ローカルで実行する分には必要ありません。

In [ ]:
import os
os.environ['BUCKET'] = BUCKET

!gsutil cp ./data/video/*.tfrecord gs://$BUCKET/data/youtube-8m/

### データセットの作成用のコード

In [8]:
import tensorflow as tf

print(tf.__version__)

1.15.0


In [9]:
tf.enable_eager_execution()

In [10]:
from data import read_dataset

In [11]:
ds = read_dataset("./data/video/train*.tfrecord", tf.estimator.ModeKeys.TRAIN, batch_size=2)

In [12]:
for row in ds.take(1):
    print(row)

({'id': <tf.Tensor: id=71, shape=(2,), dtype=string, numpy=array([b'eePZ', b'DmPZ'], dtype=object)>, 'mean_audio': <tf.Tensor: id=72, shape=(2, 128), dtype=float32, numpy=
array([[ 0.86669576, -0.7751874 , -0.7579086 , -0.53368247,  0.55297023,
         0.15699136,  0.41251042,  0.9507806 ,  0.43265572, -0.37037003,
         0.6558467 ,  0.8589721 , -1.0398635 ,  0.24083729,  0.5707268 ,
         0.1775348 ,  0.27881876,  0.8346862 , -0.96764284, -0.1853994 ,
         0.07656934, -1.1854193 ,  0.5563145 ,  0.6056825 ,  0.04432091,
        -0.49474546, -0.38963947,  0.7327652 , -0.27744678,  0.01565564,
         0.04838182, -0.53328437, -0.17257966,  0.52597713,  0.04678931,
         0.26058447, -0.40970516,  0.16065414, -1.0552312 ,  0.9691741 ,
        -0.85592794, -0.29130167, -0.5672049 ,  0.7705078 , -0.7111683 ,
        -0.05990921, -0.7342598 ,  0.12378742,  0.9503824 ,  0.21018137,
        -0.8548928 ,  0.4956397 ,  0.15022317,  0.23852813,  0.7395334 ,
        -0.04700984,  0.1